# Amigos por idade

Contar a quantidade média de amigos por idade.


<img src="https://insight.ieeeusa.org/wp-content/uploads/sites/3/2018/08/network-1200-1200x700.jpg" class="bg-primary mb-1" width="300px">

## Implementando a aplicação

A primeira coisa a fazer é criar a aplicação e a sessão do Spark.

In [2]:
from pyspark.sql import SparkSession

# A function that splits a line of input into (age, numFriends) tuples.
def parse_line(line):
    # Split by commas
    fields = line.split(",")

    # Extract the age and numFriends fields, and convert to integers
    age = int(fields[2])
    num_friends = int(fields[3])

    # Create a tuple that is our result.
    return age, num_friends

spark = (SparkSession
         .builder
         .appName("FriendsByAge")
         .getOrCreate())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Carregar cada linha dos dados de origem em um RDD

In [9]:
lines = spark.sparkContext.textFile("s3://bigdata-ifg-files/fakefriends.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Use nossa função `parseLines` para converter em tuplas (age, numFriends)

In [10]:
rdd = lines.map(parse_line)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Muita coisa acontecendo aqui...

Estamos começando com um RDD de forma (idade, numAmigos) onde idade é a CHAVE e numAmigos é o VALOR

Usamos `mapValues` para converter cada valor `numFriends` em uma tupla de (numFriends, 1)

Em seguida, usamos `reduceByKey` para somar o total de `numFriends` e o total de instâncias para cada idade, somando todos os valores numFriends e 1s respectivamente.

In [11]:
totals_by_age = rdd.mapValues(lambda x: (x, 1)).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Então agora temos tuplas de (idade, (totalFriends, totalInstances))

Para calcular a média dividimos totalFriends/totalInstances para cada idade.

In [12]:
average_by_age = totals_by_age.mapValues(lambda x: x[0] / x[1]).sortByKey()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Colete os resultados do RDD

Isso inicia a computação do DAG e realmente executa o trabalho

In [13]:
results = average_by_age.collect()
for r in results:
  print(r)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(18, 343.375)
(19, 213.27272727272728)
(20, 165.0)
(21, 350.875)
(22, 206.42857142857142)
(23, 246.3)
(24, 233.8)
(25, 197.45454545454547)
(26, 242.05882352941177)
(27, 228.125)
(28, 209.1)
(29, 215.91666666666666)
(30, 235.8181818181818)
(31, 267.25)
(32, 207.9090909090909)
(33, 325.3333333333333)
(34, 245.5)
(35, 211.625)
(36, 246.6)
(37, 249.33333333333334)
(38, 193.53333333333333)
(39, 169.28571428571428)
(40, 250.8235294117647)
(41, 268.55555555555554)
(42, 303.5)
(43, 230.57142857142858)
(44, 282.1666666666667)
(45, 309.53846153846155)
(46, 223.69230769230768)
(47, 233.22222222222223)
(48, 281.4)
(49, 184.66666666666666)
(50, 254.6)
(51, 302.14285714285717)
(52, 340.6363636363636)
(53, 222.85714285714286)
(54, 278.0769230769231)
(55, 295.53846153846155)
(56, 306.6666666666667)
(57, 258.8333333333333)
(58, 116.54545454545455)
(59, 220.0)
(60, 202.71428571428572)
(61, 256.22222222222223)
(62, 220.76923076923077)
(63, 384.0)
(64, 281.3333333333333)
(65, 298.2)
(66, 276.4444444444444